In [1]:
# On the EDITO jupyter-python-ocean-science image, geopandas is missing
# Let's install it
# !mamba install --yes --quiet geopandas

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import geopandas as gpd
import copernicusmarine
import datetime
import shapely
# import pickle
import numpy as np

# Load Sites 

In [4]:
def _load_locations():
    fname = '../data/galicia_mussel_farms.geojson'
    # gdf_aoi = gpd.read_file(fname)
    # # return gdf_aoi.head(1).geometry.bounds
    # return gdf_aoi.head(1).geometry.bounds.values.tolist()[0]
    gdf_aoi = gpd.read_file('../data/galicia_mussel_farms.geojson')
    gdf_aoi['centroid'] = gdf_aoi.geometry.to_crs(epsg=25830).centroid.to_crs(epsg=4326)
    return gdf_aoi
    
locations = _load_locations()
locations

,name,geometry,centroid
0,Ría de Arousa,"POLYGON ((-8.95 42.54, -8.75 42.54, -8.75 42.6...",POINT (-8.85 42.61002)
1,Ría de Vigo,"POLYGON ((-8.82 42.2, -8.64 42.2, -8.64 42.32,...",POINT (-8.73 42.26002)
2,Ría de Pontevedra,"POLYGON ((-8.85 42.34, -8.71 42.34, -8.71 42.4...",POINT (-8.78 42.39001)
3,Ría de Muros e Noia,"POLYGON ((-9.08 42.72, -8.85 42.72, -8.85 42.8...",POINT (-8.96501 42.79503)


In [10]:
for row in locations.itertuples():
    print(row.centroid.x)

-8.850003718013262
-8.730003342871981
-8.780001741153763
-8.96500572140616


# Load Time-Series for Sites

In [5]:
def _load_and_average_data_for_location(bounds, tlo, thi):
    xlo, ylo, xhi, yhi = bounds
    data_request = {
        "dataset_id": "cmems_mod_ibi_phy_anfc_0.027deg-2D_PT1H-m",
        "longitude" : [xlo, xhi],
        "latitude" : [ylo, yhi],
        "time" : [tlo, thi],
        "variables" : ["thetao", "zos"] 
    }
    dset = copernicusmarine.open_dataset(
        dataset_id = data_request["dataset_id"],
        minimum_longitude = data_request["longitude"][0],
        maximum_longitude = data_request["longitude"][1],
        minimum_latitude = data_request["latitude"][0],
        maximum_latitude = data_request["latitude"][1],
        start_datetime = data_request["time"][0],
        end_datetime = data_request["time"][1],
        variables = data_request["variables"]
    )
    df = dset.to_dataframe()
    df = df.groupby('time').agg({
        "thetao": "mean",
        "zos": "mean",
    })
    return df

geometry = locations.iloc[0].geometry
bounds = shapely.bounds(geometry)

# tlo = datetime.datetime.now().isoformat()
now = datetime.datetime.now()
tlo = datetime.datetime(now.year, now.month, now.day, now.hour).isoformat()
thi = (datetime.datetime.now() + datetime.timedelta(hours=3)).isoformat()

df = _load_and_average_data_for_location(bounds, tlo, thi)

INFO - 2025-05-22T08:29:53Z - Selected dataset version: "202411"
INFO - 2025-05-22T08:29:53Z - Selected dataset part: "default"


In [6]:
df

,thetao,zos
time,,
2025-05-22 10:00:00,17.167000,0.404667
2025-05-22 11:00:00,17.233000,0.423333
2025-05-22 12:00:00,17.316667,0.206000
2025-05-22 13:00:00,17.408000,-0.216667


In [7]:
def _load_and_average_data_for_all_locations(locations, tlo, thi):
    frames = {}
    for row in locations.itertuples():
        print(row.name)
        frames[row.name] = _load_and_average_data_for_location(
            shapely.bounds(row.geometry),
            tlo,
            thi,
        )
    return frames

# tlo = datetime.datetime.now().isoformat()
now = datetime.datetime.now()
tlo = datetime.datetime(now.year, now.month, now.day, now.hour).isoformat()
thi = (datetime.datetime.now() + datetime.timedelta(hours=3)).isoformat()

frames = _load_and_average_data_for_all_locations(locations, tlo, thi)

Ría de Arousa


INFO - 2025-05-22T08:30:01Z - Selected dataset version: "202411"
INFO - 2025-05-22T08:30:01Z - Selected dataset part: "default"


Ría de Vigo


INFO - 2025-05-22T08:30:05Z - Selected dataset version: "202411"
INFO - 2025-05-22T08:30:05Z - Selected dataset part: "default"


Ría de Pontevedra


INFO - 2025-05-22T08:30:09Z - Selected dataset version: "202411"
INFO - 2025-05-22T08:30:09Z - Selected dataset part: "default"


Ría de Muros e Noia


INFO - 2025-05-22T08:30:13Z - Selected dataset version: "202411"
INFO - 2025-05-22T08:30:13Z - Selected dataset part: "default"


In [8]:
for site_name, df in frames.items():
    print(site_name)
    print(df)
    print('***')

Ría de Arousa
                        thetao       zos
time                                    
2025-05-22 10:00:00  17.167000  0.404667
2025-05-22 11:00:00  17.233000  0.423333
2025-05-22 12:00:00  17.316667  0.206000
2025-05-22 13:00:00  17.408000 -0.216667
***
Ría de Vigo
                      thetao     zos
time                                
2025-05-22 10:00:00  16.7675  0.3805
2025-05-22 11:00:00  16.8540  0.3940
2025-05-22 12:00:00  16.9500  0.1675
2025-05-22 13:00:00  17.0495 -0.2440
***
Ría de Pontevedra
                     thetao    zos
time                              
2025-05-22 10:00:00  17.199  0.388
2025-05-22 11:00:00  17.280  0.405
2025-05-22 12:00:00  17.375  0.181
2025-05-22 13:00:00  17.476 -0.230
***
Ría de Muros e Noia
                        thetao       zos
time                                    
2025-05-22 10:00:00  16.355334  0.389667
2025-05-22 11:00:00  16.429667  0.425000
2025-05-22 12:00:00  16.507334  0.212667
2025-05-22 13:00:00  16.574000 -0.196333
